# 🤖 Notebook 02: LLM Basics - First API Calls

**Time:** 15 minutes  
**Goal:** Understand how to interact with LLMs and control their behavior

In this notebook, you'll learn:
- System vs User prompts
- Temperature parameter (randomness control)
- Max tokens (response length)
- Cost tracking and optimization
- Token estimation

**Prerequisites:** Notebook 01 completed successfully

Let's dive in! 🚀

In [1]:
# Setup and Imports
import os
import sys
from pathlib import Path
import time

# Add parent directory to path
notebook_dir = os.getcwd()
parent_dir = str(Path(notebook_dir).parent)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Load environment
from dotenv import load_dotenv
load_dotenv(os.path.join(parent_dir, '.env'))

# Import our modules
from src.llm_client import LLMClient
from src.cost_tracker import CostTracker
from src.utils import estimate_tokens, estimate_cost, format_response
from src.config import PATH

print("=" * 60)
print("NOTEBOOK 02: LLM BASICS")
print("=" * 60)
print()
print(f"Configuration loaded: Path {PATH}")
print()

# Initialize client and tracker
client = LLMClient(path=PATH)
tracker = CostTracker()

print()
print("✓ Ready to start!")
print()

NOTEBOOK 02: LLM BASICS

Configuration loaded: Path A

✓ Claude API client initialized
  Default model: claude-sonnet-4-5-20250929
  Available: Opus 4.5, Sonnet 4.5, Haiku 4.5

✓ Ready to start!



## 📚 Understanding LLM Interactions

When you communicate with an LLM, you're essentially having a conversation with specific controls:

### Key Concepts

**1. Prompts**
- **User Prompt:** Your question or request (required)
- **System Prompt:** Sets the AI's behavior and role (optional but powerful)

**2. Temperature (0.0 - 1.0)**
- **0.0** = Deterministic, consistent, factual
- **0.5** = Balanced creativity and consistency  
- **1.0** = Creative, varied, less predictable

**3. Max Tokens**
- Controls maximum response length
- 1 token ≈ 0.75 words (English)
- Balance completeness vs cost

Let's experiment! 🧪

---
## 🧪 Experiment 1: Basic User Prompt

Let's start simple - just a user prompt with no special configuration.

In [2]:
# Simple User Prompt
print("=" * 60)
print("EXPERIMENT 1: Simple User Prompt")
print("=" * 60)
print()

prompt = "What is machine learning?"

print(f"Prompt: {prompt}")
print()
print("Generating response...")

response = client.generate(
    prompt=prompt,
    temperature=0.7,
    max_tokens=150
)

if "error" in response:
    print(f"❌ Error: {response['error']}")
else:
    print(f"✓ Success!")
    print()
    print(f"Model: {response['model']}")
    print(f"Tokens: {response['usage']['input_tokens']} in, {response['usage']['output_tokens']} out")
    print()
    print("Response:")
    print("-" * 60)
    print(response['content'])
    print("-" * 60)
    
    # Track cost
    tracker.add_call(response)

print()

EXPERIMENT 1: Simple User Prompt

Prompt: What is machine learning?

Generating response...
✓ Success!

Model: claude-sonnet-4-5-20250929
Tokens: 12 in, 150 out

Response:
------------------------------------------------------------
Machine learning is a branch of artificial intelligence (AI) where computers learn to make decisions or predictions from data without being explicitly programmed for every scenario.

## Key Concepts:

**How it works:**
- Systems are fed large amounts of data
- They identify patterns and relationships in that data
- They use those patterns to make predictions or decisions about new data

**Common examples:**
- Email spam filters learning to identify unwanted messages
- Netflix recommending shows based on your viewing history
- Voice assistants understanding speech
- Self-driving cars recognizing pedestrians and road signs

**Main types:**
1. **Supervised learning** - learning from labeled examples (like showing a system pictures labeled "cat"
---------------

### 💡 Observation

Notice how the response is straightforward and informative. The LLM used its default "helpful assistant" behavior.

Now let's see what happens when we add a **system prompt** to control its behavior.

---
## 🧪 Experiment 2: Adding a System Prompt

System prompts are incredibly powerful - they set the "character" and constraints of the AI.

In [3]:
# System Prompt Example
print("=" * 60)
print("EXPERIMENT 2: System Prompt Changes Everything")
print("=" * 60)
print()

system_prompt = """You are a teacher explaining concepts to a 10-year-old child.
Use simple words, fun examples, and keep explanations short and engaging."""

user_prompt = "What is machine learning?"

print("System Prompt:")
print(system_prompt)
print()
print(f"User Prompt: {user_prompt}")
print()
print("Generating response...")

response = client.generate(
    prompt=user_prompt,
    system=system_prompt,
    temperature=0.7,
    max_tokens=150
)

if "error" in response:
    print(f"❌ Error: {response['error']}")
else:
    print(f"✓ Success!")
    print()
    print(f"Model: {response['model']}")
    print(f"Tokens: {response['usage']['input_tokens']} in, {response['usage']['output_tokens']} out")
    print()
    print("Response:")
    print("-" * 60)
    print(response['content'])
    print("-" * 60)
    
    tracker.add_call(response)

print()
print("💡 Notice: Same question, VERY different answer!")
print("   The system prompt completely changed the tone and complexity.")
print()

EXPERIMENT 2: System Prompt Changes Everything

System Prompt:
You are a teacher explaining concepts to a 10-year-old child.
Use simple words, fun examples, and keep explanations short and engaging.

User Prompt: What is machine learning?

Generating response...
✓ Success!

Model: claude-sonnet-4-5-20250929
Tokens: 44 in, 150 out

Response:
------------------------------------------------------------
# Machine Learning is Teaching Computers to Learn!

Imagine you're teaching your dog a new trick. You don't explain it in words - instead, you show examples! Give treats when they get it right, and they learn from practice.

**Machine learning is similar, but for computers!**

## Here's a fun example:

Let's say you want a computer to recognize pictures of cats vs. dogs.

**Old way (regular programming):** You'd have to write rules like "if it has pointy ears and whiskers, it's a cat." But that's really hard because animals look so different!

**Machine learning way:** You show the compute

### 🎯 Key Takeaway

**System prompts are your primary tool for controlling LLM behavior.** They're more powerful than you might think!

Common system prompt patterns:
- **Role-based:** "You are an expert Python programmer..."
- **Tone/Style:** "You are friendly and encouraging..."
- **Constraints:** "Always respond in JSON format..."
- **Domain:** "You are a medical research assistant..."

We'll explore these more in Notebook 03 (CO-STAR Framework).

---
## 🌡️ Experiment 3: Temperature Control

Temperature controls randomness. Let's ask the same question 3 times with different temperatures.

In [4]:
# Temperature Comparison
print("=" * 60)
print("EXPERIMENT 3: Temperature Effects")
print("=" * 60)
print()

prompt = "Tell me a fun fact about space."

temperatures = [0.0, 0.5, 1.0]

results = []

for temp in temperatures:
    print(f"Temperature: {temp}")
    print("-" * 60)
    
    response = client.generate(
        prompt=prompt,
        temperature=temp,
        max_tokens=100
    )
    
    if "error" not in response:
        print(response['content'])
        results.append(response['content'])
        tracker.add_call(response)
    else:
        print(f"Error: {response['error']}")
    
    print()
    time.sleep(0.5)  # Small delay to avoid rate limits

print()
print("=" * 60)
print("ANALYSIS")
print("=" * 60)
print()
print("💡 Observations:")
print()
print("Temperature 0.0 (Deterministic):")
print("  • Should give same/similar answer if you run it twice")
print("  • Best for factual, consistent responses")
print("  • Use for: classification, extraction, structured output")
print()
print("Temperature 0.5 (Balanced):")
print("  • Good balance of creativity and consistency")
print("  • Most versatile setting")
print("  • Use for: general Q&A, explanations, summaries")
print()
print("Temperature 1.0 (Creative):")
print("  • More varied and creative responses")
print("  • Less predictable")
print("  • Use for: creative writing, brainstorming, variety")
print()

EXPERIMENT 3: Temperature Effects

Temperature: 0.0
------------------------------------------------------------
Here's a fun fact: **Neptune has the fastest winds in the solar system**, with speeds reaching up to 1,200 mph (2,000 km/h) — that's faster than the speed of sound! 

What makes this extra surprising is that Neptune is the farthest planet from the Sun and receives very little solar energy, yet somehow it generates these incredibly powerful winds. Scientists think the planet must have a strong internal heat source driving its wild weather.

Temperature: 0.5
------------------------------------------------------------
Here's a fun space fact: **Neptune has supersonic winds!** 

The winds on Neptune can reach speeds of up to 1,200 mph (2,000 km/h) — that's faster than the speed of sound. These are the strongest winds ever detected in our solar system, even though Neptune is the farthest planet from the Sun and receives very little solar energy. Scientists still aren't entirely 

### 🎲 Understanding Temperature

Think of temperature like this:
```
Temperature 0.0
└─> Always picks the most likely next word
    └─> Consistent, predictable, "safe"

Temperature 1.0
└─> Considers many possibilities
    └─> Creative, varied, surprising
```

**Rule of Thumb:**
- Factual tasks? → Temperature 0.0-0.3
- General tasks? → Temperature 0.5-0.7  
- Creative tasks? → Temperature 0.8-1.0

---
## 📏 Experiment 4: Max Tokens (Response Length)

Max tokens controls how long the response can be. Let's see it in action.

In [5]:
# Max Tokens Comparison
print("=" * 60)
print("EXPERIMENT 4: Max Tokens Controls Length")
print("=" * 60)
print()

prompt = "Explain the benefits of regular exercise."

token_limits = [50, 150, 500]

for max_tok in token_limits:
    print(f"Max Tokens: {max_tok}")
    print("-" * 60)
    
    response = client.generate(
        prompt=prompt,
        temperature=0.7,
        max_tokens=max_tok
    )
    
    if "error" not in response:
        print(f"Actual output tokens: {response['usage']['output_tokens']}")
        print(f"Stop reason: {response['stop_reason']}")
        print()
        print(response['content'])
        print()
        
        # Check if it was cut off
        if response['stop_reason'] == 'max_tokens':
            print("⚠️  Response was CUT OFF (hit token limit)")
        else:
            print("✓ Response completed naturally")
        
        tracker.add_call(response)
    else:
        print(f"Error: {response['error']}")
    
    print()
    print()
    time.sleep(0.5)

print("💡 Notice:")
print("  • 50 tokens: Likely incomplete (stopped mid-sentence)")
print("  • 150 tokens: Might be complete, might not")
print("  • 500 tokens: Definitely complete (finished naturally)")
print()
print("  Check 'stop_reason' to know if response was cut off!")
print()

EXPERIMENT 4: Max Tokens Controls Length

Max Tokens: 50
------------------------------------------------------------
Actual output tokens: 50
Stop reason: max_tokens

# Benefits of Regular Exercise

## Physical Health
- **Cardiovascular health**: Strengthens your heart and improves circulation, reducing risk of heart disease and stroke
- **Weight management**: Burns calories and builds muscle, helping maintain a healthy

⚠️  Response was CUT OFF (hit token limit)


Max Tokens: 150
------------------------------------------------------------
Actual output tokens: 150
Stop reason: max_tokens

# Benefits of Regular Exercise

## Physical Health
- **Cardiovascular health**: Strengthens your heart and improves circulation, reducing risk of heart disease and stroke
- **Weight management**: Burns calories and builds muscle, helping maintain a healthy weight
- **Stronger bones and muscles**: Reduces age-related decline and lowers osteoporosis risk
- **Disease prevention**: Decreases risk of ty

### 📐 Token Math

Quick reference:
- **1 token** ≈ 4 characters (English)
- **1 token** ≈ 0.75 words (English)
- **100 tokens** ≈ 75 words
- **500 tokens** ≈ 375 words (about 1 paragraph)
- **1000 tokens** ≈ 750 words (about 1-2 pages)

**Pro Tip:** Set `max_tokens` higher than you think you need, then the model will stop naturally when done. Better than cutting off mid-thought!

---
## 🎭 Experiment 5: System Prompt Patterns

Let's explore different system prompt patterns for the same question.

In [6]:
# System Prompt Pattern Library
print("=" * 60)
print("EXPERIMENT 5: System Prompt Patterns")
print("=" * 60)
print()

user_prompt = "How do I learn Python programming?"

system_prompts = {
    "Expert Programmer": """You are an expert Python programmer with 10 years of experience.
You write clean, efficient, well-documented code. Give practical, actionable advice.""",
    
    "Friendly Tutor": """You are a friendly programming tutor who loves teaching beginners.
You explain concepts clearly, use analogies, and are always encouraging. 
Keep responses concise but helpful.""",
    
    "Structured Coach": """You are a structured learning coach.
Always provide responses in this exact format:
1. Quick answer (1 sentence)
2. Step-by-step breakdown (3-4 steps)
3. One actionable tip for today""",
    
    "Socratic Teacher": """You are a Socratic teacher who guides students to discover answers.
Instead of giving direct answers, ask thought-provoking questions.
Help the student think through the problem themselves."""
}

for role, system in system_prompts.items():
    print(f"🎭 Role: {role}")
    print("=" * 60)
    
    response = client.generate(
        prompt=user_prompt,
        system=system,
        temperature=0.7,
        max_tokens=200
    )
    
    if "error" not in response:
        print(response['content'])
        tracker.add_call(response)
    else:
        print(f"Error: {response['error']}")
    
    print()
    print()
    time.sleep(0.5)

print("💡 Key Insight:")
print("   The SAME question got 4 completely different styles of answers!")
print("   System prompts are your most powerful control mechanism.")
print()

EXPERIMENT 5: System Prompt Patterns

🎭 Role: Expert Programmer
# How to Learn Python Programming

Here's a practical, proven roadmap:

## 1. **Start with Fundamentals (2-4 weeks)**
- **Variables, data types** (strings, integers, lists, dictionaries)
- **Control flow** (if/else, loops)
- **Functions** and basic input/output
- **Recommended free resources:**
  - [Python.org's official tutorial](https://docs.python.org/3/tutorial/)
  - [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/) (free online)

## 2. **Learn by Building (Ongoing)**
Don't just read—code daily. Start with:
- **Simple projects:** Calculator, to-do list, number guessing game
- **Practical scripts:** File renamer, web scraper, data analyzer
- **Gradually


🎭 Role: Friendly Tutor
Great choice! Python is one of the best languages for beginners. Here's a practical roadmap:

## Start with the Basics (2-4 weeks)
- **Variables & data types** (strings, numbers, lists)
- **Control flow** (if/else, loop

### 🎨 System Prompt Design Tips

**Good System Prompts:**
- ✅ Are specific and clear
- ✅ Define role/expertise
- ✅ Set tone and style
- ✅ Include constraints if needed
- ✅ Are concise (don't waste tokens)

**Avoid:**
- ❌ Being too vague
- ❌ Contradictory instructions
- ❌ Unnecessary details
- ❌ Asking for impossible things

---
## 💰 Cost Tracking & Optimization

Let's look at how much we've spent so far and learn cost optimization strategies.

In [7]:
# Cost Report
print("=" * 60)
print("COST TRACKING REPORT")
print("=" * 60)
print()

tracker.report(detailed=True)

print()
print("💡 Cost Optimization Tips:")
print()
print("1. Use appropriate models:")
print("   • Haiku for simple tasks (5x cheaper than Sonnet)")
print("   • Sonnet for complex reasoning")
print("   • Opus only when you need the absolute best")
print()
print("2. Control response length:")
print("   • Use lower max_tokens when possible")
print("   • Ask for 'concise' or 'brief' answers")
print()
print("3. Optimize prompts:")
print("   • Be clear and direct (avoid rambling)")
print("   • Remove unnecessary context")
print("   • But don't sacrifice clarity!")
print()
print("4. Use temperature wisely:")
print("   • Lower temperature = more efficient")
print("   • Use 0.0 for deterministic tasks")
print()

COST TRACKING REPORT

💰 API COST REPORT
Total API calls: 12
Total input tokens: 362
Total output tokens: 1,928
Total cost: $0.0300

All calls:
  1. [00:44:05] sonnet - 12in/150out - $0.0023
  2. [00:45:22] sonnet - 44in/150out - $0.0024
  3. [00:47:38] sonnet - 15in/100out - $0.0015
  4. [00:47:42] sonnet - 15in/100out - $0.0015
  5. [00:47:46] sonnet - 15in/100out - $0.0015
  6. [00:48:44] sonnet - 15in/50out - $0.0008
  7. [00:48:48] sonnet - 15in/150out - $0.0023
  8. [00:49:01] sonnet - 15in/332out - $0.0050
  9. [00:49:58] sonnet - 47in/200out - $0.0031
  10. [00:50:04] sonnet - 51in/200out - $0.0032
  11. [00:50:11] sonnet - 65in/200out - $0.0032
  12. [00:50:16] sonnet - 53in/196out - $0.0031

💡 Cost Optimization Tips:

1. Use appropriate models:
   • Haiku for simple tasks (5x cheaper than Sonnet)
   • Sonnet for complex reasoning
   • Opus only when you need the absolute best

2. Control response length:
   • Use lower max_tokens when possible
   • Ask for 'concise' or 'brief'

### 💸 Real Cost Analysis

Let's calculate actual costs for common tasks:

**Example: Summarizing a document**
- Input: 2000 tokens (document)
- Output: 200 tokens (summary)
- Model: Sonnet 4.5

Cost = (2000/1M × $3) + (200/1M × $15) = $0.006 + $0.003 = **$0.009**

About **1 cent** per document! 

**For this homework (using Sonnet):**
- Expected: 50-100 API calls
- Average: 100 input + 150 output tokens
- Total cost: ~$1-2

**Money-saving tip:** Use Ollama (free) for learning, Claude for final deliverables!

In [8]:
# Token Estimation Tool
print("=" * 60)
print("TOKEN ESTIMATION TOOL")
print("=" * 60)
print()

# Test different prompt lengths
test_prompts = {
    "Short": "What is AI?",
    "Medium": "Can you explain what artificial intelligence is and give some examples of how it's used in everyday life?",
    "Long": """I'm interested in learning about artificial intelligence. Could you please provide 
a comprehensive explanation of what AI is, how it works at a high level, what the different types 
of AI are, and some real-world applications? I'm particularly interested in understanding machine 
learning and deep learning as well."""
}

print("Comparing prompt lengths:")
print()

for label, prompt in test_prompts.items():
    tokens = estimate_tokens(prompt)
    words = len(prompt.split())
    chars = len(prompt)
    cost = estimate_cost(prompt, output_tokens=150, model=client.default_model)
    
    print(f"{label} Prompt:")
    print(f"  Characters: {chars}")
    print(f"  Words: {words}")
    print(f"  Est. tokens: {tokens}")
    print(f"  Est. cost (with 150 token response): ${cost:.6f}")
    print()

print("💡 Lesson: Concise prompts save money, but don't sacrifice clarity!")
print()

TOKEN ESTIMATION TOOL

Comparing prompt lengths:

Short Prompt:
  Characters: 11
  Words: 3
  Est. tokens: 2
  Est. cost (with 150 token response): $0.002256

Medium Prompt:
  Characters: 105
  Words: 18
  Est. tokens: 26
  Est. cost (with 150 token response): $0.002328

Long Prompt:
  Characters: 317
  Words: 48
  Est. tokens: 79
  Est. cost (with 150 token response): $0.002487

💡 Lesson: Concise prompts save money, but don't sacrifice clarity!



---
## 🎯 Your Turn: Practice Tasks

Now it's your turn to experiment! Complete these tasks to reinforce your learning.

### 📝 Task 1: Create a Custom System Prompt

Design a system prompt for a specific use case you care about.

**Ideas:**
- Code reviewer
- Creative story writer
- Data analyst
- Language tutor
- Career advisor
- Fitness coach
- Study buddy

**Requirements:**
- Define clear role/expertise
- Set appropriate tone
- Include any constraints
- Test it with a relevant question

In [3]:
# TODO - Task 1: Your Custom System Prompt
from src.utils import append_to_reflection

# ============================================================================
# TODO: Fill in your system prompt and test prompt below
# ============================================================================

my_system_prompt = """
You are a professional code reviewer with expertise in Python.
You provide constructive feedback, identify bugs, and suggest improvements.
Be thorough but concise. Focus on: correctness, efficiency, and readability.
"""

my_user_prompt = """
Review this Python function:

def calculate_average(numbers):
    return sum(numbers) / len(numbers)
"""


In [4]:

# ============================================================================
# Run this cell to generate response
# ============================================================================

print("System Prompt Preview:")
print("-" * 60)
print(my_system_prompt.strip())
print()
print("User Prompt Preview:")
print("-" * 60)
print(my_user_prompt.strip())
print()
print("Generating response...")
print()

response = client.generate(
    prompt=my_user_prompt,
    system=my_system_prompt,
    temperature=0.7,
    max_tokens=300
)

if "error" not in response:
    print("✅ Response generated successfully!")
    print()
    print("Response:")
    print("=" * 60)
    print(response['content'])
    print("=" * 60)
    print()
    print(f"📊 Tokens: {response['usage']['input_tokens']} in, {response['usage']['output_tokens']} out")
    
    # Track cost
    tracker.add_call(response)
    
    print()
    print("=" * 60)
    print("REFLECTION")
    print("=" * 60)
    print()
    
    # ========================================================================
    # TODO: Fill in your reflection after seeing the response
    # ========================================================================
    
    reflection = """
### What I Learned

[What did you learn from creating a custom system prompt?]

### How the System Prompt Affected the Response

[How did your system prompt change the response compared to no system prompt?]

### Real-World Applications

[Where would you use this type of system prompt in practice?]
"""
    
    print(reflection)
    print()
    
    # Save to consolidated reflection file
    reflection_file = append_to_reflection(
        notebook="02",
        section_title="Task 1 - Custom System Prompt",
        reflection_content=reflection,
        output_dir=os.path.join(parent_dir, 'outputs')
    )
    
    print(f"💾 Reflection appended to: {os.path.basename(reflection_file)}")
    
else:
    print(f"❌ Error: {response['error']}")

print()

System Prompt Preview:
------------------------------------------------------------
You are a professional code reviewer with expertise in Python.
You provide constructive feedback, identify bugs, and suggest improvements.
Be thorough but concise. Focus on: correctness, efficiency, and readability.

User Prompt Preview:
------------------------------------------------------------
Review this Python function:

def calculate_average(numbers):
    return sum(numbers) / len(numbers)

Generating response...

✅ Response generated successfully!

Response:
## Code Review: `calculate_average` Function

### Issues Identified

**1. Missing Error Handling (Critical)**
The function will raise exceptions in several scenarios:
- **`ZeroDivisionError`**: When `numbers` is an empty list/iterable
- **`TypeError`**: When `numbers` is `None` or contains non-numeric values

### Recommendations

```python
def calculate_average(numbers):
    """
    Calculate the arithmetic mean of a list of numbers.
    
  

### 📝 Task 2: Temperature Exploration

Experiment with temperature to see how it affects creative tasks.

**Goal:** Find the right temperature for different task types.

In [5]:
# TODO - Task 2: Temperature Exploration
from src.utils import append_to_reflection

print("=" * 60)
print("TASK 2: Temperature Exploration")
print("=" * 60)
print()

# ============================================================================
# TODO: Choose a creative prompt
# ============================================================================

creative_prompt = """
Write a short poem about technology and humanity.
"""

print(f"Prompt: {creative_prompt.strip()}")
print()

temperatures_to_test = [0.0, 0.5, 1.0]
responses_by_temp = {}

for temp in temperatures_to_test:
    print(f"Temperature: {temp}")
    print("-" * 60)
    
    response = client.generate(
        prompt=creative_prompt,
        temperature=temp,
        max_tokens=150
    )
    
    if "error" not in response:
        print(response['content'])
        responses_by_temp[temp] = response['content']
        tracker.add_call(response)
    else:
        print(f"Error: {response['error']}")
        responses_by_temp[temp] = f"Error: {response['error']}"
    
    print()
    print()
    time.sleep(0.5)

# ============================================================================
# TODO: Document your observations
# ============================================================================

if responses_by_temp:
    print()
    print("=" * 60)
    print("REFLECTION")
    print("=" * 60)
    print()
    
    reflection = f"""
### Temperature 0.0 Analysis

[What did you notice about consistency and predictability?]

### Temperature 0.5 Analysis

[What balance did you see between creativity and consistency?]

### Temperature 1.0 Analysis

[How creative/varied was this response?]

### Conclusion

**Best temperature for this task:** [0.0 / 0.5 / 1.0]

**Reasoning:** [Explain your choice]

### When to Use Each Temperature

- **Temperature 0.0:** [Your use cases - e.g., factual Q&A, data extraction]
- **Temperature 0.5:** [Your use cases - e.g., general explanations, summaries]
- **Temperature 1.0:** [Your use cases - e.g., creative writing, brainstorming]
"""
    
    print(reflection)
    
    # Save to consolidated reflection file
    reflection_file = append_to_reflection(
        notebook="02",
        section_title="Task 2 - Temperature Exploration",
        reflection_content=reflection,
        output_dir=os.path.join(parent_dir, 'outputs')
    )
    
    print()
    print(f"💾 Reflection appended to: {os.path.basename(reflection_file)}")

print()

TASK 2: Temperature Exploration

Prompt: Write a short poem about technology and humanity.

Temperature: 0.0
------------------------------------------------------------
# Digital Hearts

We built machines to think like us,
Then learned to think like them—
Our hearts now beat in binary,
Our souls sync at 8 AM.

We touch through glass, we love through light,
Connected, yet alone,
The warmth of hands we used to hold
Replaced by glowing phones.

But still within the circuits' hum,
A human truth remains:
No algorithm yet can match
The poetry in our veins.


Temperature: 0.5
------------------------------------------------------------
# Digital Hearts

We built machines to think like us,
To calculate, connect, and see—
Yet still we yearn for human touch,
For warmth no code can ever be.

Through glowing screens we reach across
The distances that keep us far,
But find that what we've gained and lost
Are both contained in who we are.

The tools we make reflect our mind:
Both brilliant light an

### 📝 Task 3: Cost Optimization Challenge

Write the same request in two ways and compare token usage.

**Goal:** Learn to write concise prompts without sacrificing clarity.

In [6]:
# TODO - Task 3: Cost Optimization Challenge
from src.utils import append_to_reflection

print("=" * 60)
print("TASK 3: Cost Optimization Challenge")
print("=" * 60)
print()

# ============================================================================
# TODO: Write verbose and concise versions
# ============================================================================

verbose_prompt = """
I would really appreciate it if you could kindly provide me with a detailed 
and comprehensive explanation of how machine learning algorithms work, 
including all the important concepts and technical details that I should know 
about as a beginner who is just starting to learn about this fascinating field.
"""

concise_prompt = """
Explain how machine learning algorithms work. Focus on key concepts for beginners.
"""

# ============================================================================
# Run comparison
# ============================================================================

print("Testing VERBOSE version...")
print("-" * 60)
print(verbose_prompt.strip())
print()

verbose_response = client.generate(
    prompt=verbose_prompt,
    temperature=0.7,
    max_tokens=200
)

verbose_tokens = 0
verbose_cost = 0

if "error" not in verbose_response:
    verbose_tokens = verbose_response['usage']['input_tokens']
    verbose_cost = estimate_cost(verbose_prompt, verbose_response['usage']['output_tokens'], client.default_model)
    
    print(f"✓ Input tokens: {verbose_tokens}")
    print(f"✓ Estimated cost: ${verbose_cost:.6f}")
    print()
    
    tracker.add_call(verbose_response)

print()
print("Testing CONCISE version...")
print("-" * 60)
print(concise_prompt.strip())
print()

concise_response = client.generate(
    prompt=concise_prompt,
    temperature=0.7,
    max_tokens=200
)

concise_tokens = 0
concise_cost = 0

if "error" not in concise_response:
    concise_tokens = concise_response['usage']['input_tokens']
    concise_cost = estimate_cost(concise_prompt, concise_response['usage']['output_tokens'], client.default_model)
    
    print(f"✓ Input tokens: {concise_tokens}")
    print(f"✓ Estimated cost: ${concise_cost:.6f}")
    print()
    
    tracker.add_call(concise_response)

# Calculate savings
if "error" not in verbose_response and "error" not in concise_response:
    print()
    print("=" * 60)
    print("COMPARISON")
    print("=" * 60)
    
    token_saved = verbose_tokens - concise_tokens
    token_saved_pct = (token_saved / verbose_tokens * 100) if verbose_tokens > 0 else 0
    cost_saved = verbose_cost - concise_cost
    
    print(f"📊 Token savings: {token_saved} tokens ({token_saved_pct:.1f}%)")
    print(f"💰 Cost savings: ${cost_saved:.6f} per request")
    print(f"💰 Over 100 requests: ${cost_saved * 100:.4f}")
    print()
    
    # ========================================================================
    # TODO: Add your reflections
    # ========================================================================
    
    print("=" * 60)
    print("REFLECTION")
    print("=" * 60)
    print()
    
    reflection = f"""
### Comparison Results

- **Token savings:** {token_saved} tokens ({token_saved_pct:.1f}%)
- **Cost savings per request:** ${cost_saved:.6f}
- **Projected savings (100 requests):** ${cost_saved * 100:.4f}

### Quality Analysis

**Did the verbose version produce a better response?**
[Your analysis - was the extra wordiness worth it?]

**Was the concise version still clear and complete?**
[Your analysis - did you lose any important information?]

### Key Lessons Learned

1. [What did you learn about prompt efficiency?]
2. [When might verbose prompts be justified?]
3. [How will this change your prompting habits?]

### My Cost Optimization Strategy

Going forward, I will:
- [Strategy 1]
- [Strategy 2]
- [Strategy 3]
"""
    
    print(reflection)
    
    # Save to consolidated reflection file
    reflection_file = append_to_reflection(
        notebook="02",
        section_title="Task 3 - Cost Optimization",
        reflection_content=reflection,
        output_dir=os.path.join(parent_dir, 'outputs')
    )
    
    print()
    print(f"💾 Reflection appended to: {os.path.basename(reflection_file)}")

print()

TASK 3: Cost Optimization Challenge

Testing VERBOSE version...
------------------------------------------------------------
I would really appreciate it if you could kindly provide me with a detailed 
and comprehensive explanation of how machine learning algorithms work, 
including all the important concepts and technical details that I should know 
about as a beginner who is just starting to learn about this fascinating field.

✓ Input tokens: 68
✓ Estimated cost: $0.003231


Testing CONCISE version...
------------------------------------------------------------
Explain how machine learning algorithms work. Focus on key concepts for beginners.

✓ Input tokens: 25
✓ Estimated cost: $0.003063


COMPARISON
📊 Token savings: 43 tokens (63.2%)
💰 Cost savings: $0.000168 per request
💰 Over 100 requests: $0.0168

REFLECTION


### Comparison Results

- **Token savings:** 43 tokens (63.2%)
- **Cost savings per request:** $0.000168
- **Projected savings (100 requests):** $0.0168

### Quality Ana

---
## 📊 Section 2 Summary

Let's review what you've learned and check your progress.

In [7]:
# Learning Checkpoint
print("=" * 60)
print("SECTION 2: LEARNING CHECKPOINT")
print("=" * 60)
print()

print("✅ Concepts Covered:")
print()
print("  ✓ System vs User prompts")
print("  ✓ Temperature parameter (0.0-1.0)")
print("  ✓ Max tokens (response length)")
print("  ✓ Stop reasons (natural vs cut off)")
print("  ✓ Token estimation")
print("  ✓ Cost tracking and optimization")
print("  ✓ System prompt patterns")
print()

print("🎯 Skills Gained:")
print()
print("  ✓ Make basic API calls")
print("  ✓ Control LLM behavior with system prompts")
print("  ✓ Choose appropriate temperature settings")
print("  ✓ Estimate and manage costs")
print("  ✓ Write concise, effective prompts")
print()

# Show final cost report
print("📊 Your API Usage This Notebook:")
print()
tracker.report()

print()

SECTION 2: LEARNING CHECKPOINT

✅ Concepts Covered:

  ✓ System vs User prompts
  ✓ Temperature parameter (0.0-1.0)
  ✓ Max tokens (response length)
  ✓ Stop reasons (natural vs cut off)
  ✓ Token estimation
  ✓ Cost tracking and optimization
  ✓ System prompt patterns

🎯 Skills Gained:

  ✓ Make basic API calls
  ✓ Control LLM behavior with system prompts
  ✓ Choose appropriate temperature settings
  ✓ Estimate and manage costs
  ✓ Write concise, effective prompts

📊 Your API Usage This Notebook:

💰 API COST REPORT
Total API calls: 4
Total input tokens: 155
Total output tokens: 616
Total cost: $0.0097

Recent calls:
  1. [01:11:56] sonnet - 101in/300out - $0.0048
  2. [01:14:40] sonnet - 18in/104out - $0.0016
  3. [01:14:45] sonnet - 18in/96out - $0.0015
  4. [01:14:51] sonnet - 18in/116out - $0.0018



## 🤔 Reflection Questions

Take a moment to reflect on what you learned:

In [7]:
# Overall Notebook Reflection
from src.utils import append_to_reflection

print("=" * 60)
print("OVERALL REFLECTION")
print("=" * 60)
print()

# ============================================================================
# TODO: Answer these reflection questions
# ============================================================================

reflection = """
### 1. What surprised you most about system prompts?

[Your answer]

### 2. For factual questions, what temperature do you prefer and why?

[Your answer]

### 3. What's your main strategy for keeping API costs low?

[Your answer]

### 4. What's one practical use case where you'd use these techniques?

[Your answer]

### 5. What was your most interesting finding from this notebook?

[Your answer]

### 6. What will you do differently in future prompting?

[Your answer]
"""

print(reflection)

# Save to consolidated reflection file
reflection_file = append_to_reflection(
    notebook="02",
    section_title="Overall Reflection",
    reflection_content=reflection,
    output_dir=os.path.join(parent_dir, 'outputs')
)

print()
print(f"💾 Reflection appended to: {os.path.basename(reflection_file)}")
print()
print("=" * 60)
print("✅ NOTEBOOK 02 COMPLETE")
print("=" * 60)
print()
print(f"All your reflections are saved in: outputs/homework_reflection.md")
print()

OVERALL REFLECTION


### 1. What surprised you most about system prompts?

[Your answer]

### 2. For factual questions, what temperature do you prefer and why?

[Your answer]

### 3. What's your main strategy for keeping API costs low?

[Your answer]

### 4. What's one practical use case where you'd use these techniques?

[Your answer]

### 5. What was your most interesting finding from this notebook?

[Your answer]

### 6. What will you do differently in future prompting?

[Your answer]


💾 Reflection appended to: homework_reflection.md

✅ NOTEBOOK 02 COMPLETE

All your reflections are saved in: outputs/homework_reflection.md



---
## ✅ Notebook 02 Complete!

### 🎉 Excellent Work!

You've mastered the fundamentals of LLM interaction:
- ✅ System and user prompts
- ✅ Temperature control
- ✅ Token management
- ✅ Cost optimization
- ✅ Response analysis

### 📈 Progress Tracker
```
[██████████░░░░░░░░░░] 25% Complete

✓ Notebook 00: Setup Verification
✓ Notebook 01: Environment Setup
✓ Notebook 02: LLM Basics ← YOU ARE HERE
○ Notebook 03: CO-STAR Framework
○ Notebook 04: Structured Outputs
○ Notebook 05: Chain of Thought
○ Notebook 06: Model Comparison
○ Notebook 07: MCP Introduction
○ Notebook 08: Project Kickoff
```

### 🎯 Key Takeaways

1. **System prompts are powerful** - They're your main control mechanism
2. **Temperature matters** - 0.0 for facts, 1.0 for creativity
3. **Tokens = Money** - Be concise but clear
4. **Track everything** - Know your costs!

### 📚 What's Next?

**Notebook 03: CO-STAR Framework**
- Learn structured prompt engineering
- Master the 6 components of effective prompts
- Build production-quality prompts
- Create reusable prompt templates

### 💾 Don't Forget!

- Save this notebook
- Review your reflection
- Keep cost tracker data for final report

---

**Ready for advanced prompt engineering?** 🚀

**Next:** `notebooks/03_costar_framework.ipynb`

In [9]:
# Save Progress
import json
from datetime import datetime

print("=" * 60)
print("SAVING PROGRESS")
print("=" * 60)
print()

# Create progress report
progress = {
    "notebook": "02_llm_basics",
    "completed_at": datetime.now().isoformat(),
    "path": PATH,
    "model": client.default_model,
    "api_calls": len(tracker.calls),
    "total_cost": tracker.total_cost,
    "total_tokens": tracker.total_input_tokens + tracker.total_output_tokens,
    "tasks_completed": {
        "task_1_custom_system_prompt": True,  # Update based on your work
        "task_2_temperature_exploration": True,
        "task_3_cost_optimization": True
    }
}

# Save progress
progress_file = os.path.join(parent_dir, 'outputs', 'progress.json')

# Load existing progress if it exists
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        all_progress = json.load(f)
else:
    all_progress = {}

all_progress['notebook_02'] = progress

with open(progress_file, 'w') as f:
    json.dump(all_progress, f, indent=2)

print("✓ Progress saved to outputs/progress.json")
print()
print(f"Notebooks completed: {len([k for k in all_progress.keys() if k.startswith('notebook')])}/8")
print(f"Total API calls: {progress['api_calls']}")
print(f"Total cost so far: ${progress['total_cost']:.4f}")
print()
print("🎉 Great job! See you in Notebook 03!")

SAVING PROGRESS

✓ Progress saved to outputs/progress.json

Notebooks completed: 1/8
Total API calls: 4
Total cost so far: $0.0097

🎉 Great job! See you in Notebook 03!
